## Loading the data into Starfish

In [ ]:
from starfish.experiment import Experiment
from starfish import ImageStack
from starfish.constants import Indices
import os

This notebook demonstrates how to load osmFISH data into starfish. Below loads fov_001, however fovs 002 and 003 are also converted and can be loaded by exchanging the number in the cloudflare link. The data can be dumped for local loading with `s.image.write`

In [ ]:
experiment = Experiment.from_json('https://dmf0bdeheu4zf.cloudfront.net/20180802/osmFISH/fov_001/experiment.json')

The below plot displays the z-volume for channel 0 of fov_001

In [ ]:
experiment.show_stack({Indices.CH: 0})